# Le code suivant représente une approche d'algorithme génétique pour la planification de séances dans des intervalles de temps prédéfinis. Voici les principaux composants , le but c'est de créer des emplois du temps ou il n'ya pas de chevauchement entre les seances tout en assurant de profiter de toutes les seances disponibles :

# 1. **Constantes** :
  - `POPULATION_SIZE` : Le nombre d'individus dans chaque population.
  - `GENOME_LENGTH` : La longueur du chromosome, représentant le nombre de séances à planifier.
  - `MAX_GENERATIONS` : Le nombre maximal de générations pour exécuter l'algorithme génétique.
  - `MUTATION_RATE` : La probabilité de mutation $P(mutation)$ pour chaque gène lors de l'opération de mutation.


In [55]:
import random

# Constants
POPULATION_SIZE = 100
GENOME_LENGTH = 6
MAX_GENERATIONS = 20
MUTATION_RATE = 0.1





# 2. **Séances** :
   - La liste `sessions` définit les intervalles de temps disponibles pour la planification des séances. Chaque séance est représentée par une heure de début et une heure de fin, on definit les six seances de travail usuelles , de $08:00$ à $20:00$ , $2$ heures pour chaque session.


In [56]:
# Definir les session comme intervales [debut, fin]
sessions = [
    [8, 10],
    [10, 12],
    [12, 14],
    [14, 16],
    [16, 18],
    [18, 20]
]

# 3. **Initialisation de la population** :
   - La fonction `generate_population` crée une population initiale d'individus (chromosomes). Chaque chromosome represente un emplois du temps sous forme d'une liste à 6 elements puisqu'on a six seances où chaque gène représente une séance et peut prendre des valeurs de 0 à 2. 0 represente que la session est vide , 1 indique que la periode est occupé par une seule séances tandis que 2 indique un chevauchement de 2 séances:


In [ ]:
def generate_population():
    population = []
    for _ in range(POPULATION_SIZE):
        chromosome = [random.randint(0, 2) for _ in range(GENOME_LENGTH)]
        population.append(chromosome)
    return population

# 4. **Évaluation de la fonction de fitness** :
   - La fonction `evaluate_fitness` calcule le score de fitness pour un chromosome donné(un emploi du temps). Elle évalue le chromosome en comptant le nombre de séances qui se chevauchent et le nombre de séances vides. s'il y'a un chevauchement , on penalise l'emplois par ajouter le nombre des seances chevauchante $1 - chromosome[i]$ au score , on pénalise de plus les sessions vides par ajouter un 1 au score puisqu'on veut profiter le maximum possible de toutes les seances. un emploi idéal sera donc $[1,1,1,1,1,1]$ puisque son score est 0 ,et comme exemple le score de $[2,2,0,0,1,1]$ est 4 .
   - notre but est de minimiser le $schedule\_score$


In [ ]:
def evaluate_fitness(chromosome):
    schedule_score = 0
    for gene in chromosome:
        # Reduce the overlap of sessions
        if gene > 1:
            schedule_score += gene - 1
        # Count empty sessions
        elif gene == 0:
            schedule_score += 1
    return schedule_score

# 5. **Sélection des parents** :
   - La fonction `select_parents` sélectionne les individus (chromosomes) les plus performants de la population pour devenir des parents lors de l'opération de crossover. La sélection est basée sur leurs scores de fitness , du coup la prochaine generation aura probablement des meilleurs `score`.


In [ ]:
def select_parents(population):
    # Select the best 50% of the population as parents
    sorted_population = sorted(population, key=lambda chromosome: evaluate_fitness(chromosome))
    parents = sorted_population[:POPULATION_SIZE // 2]
    return parents

# 6. **Crossover** :
   - La fonction `crossover` effectue l'opération de crossover en combinant les chromosomes des parents sélectionnés. Elle crée deux nouveaux chromosomes (enfants) en échangeant les parties du chromosome des parents à un point de crossover aléatoire.


In [ ]:
def crossover(parent1, parent2):
    crossover_point = random.randint(1, GENOME_LENGTH - 1)
    offspring1 = parent1[:crossover_point] + parent2[crossover_point:]
    offspring2 = parent2[:crossover_point] + parent1[crossover_point:]
    return offspring1, offspring2

# 7. **Mutation** :
   - La fonction `mutate` effectue l'opération de mutation en inversant les bits (gènes) du chromosome avec une probabilité définie par `MUTATION_RATE`.


In [ ]:
def mutate(chromosome):
    for i in range(len(chromosome)):
        if random.random() < MUTATION_RATE:
            chromosome[i] = 2 - chromosome[i]

# 8. **Algorithme génétique** :
   - La fonction `genetic_algorithm` exécute l'algorithme génétique en générant une population initiale, en évaluant la fitness de chaque chromosome, en sélectionnant les parents, en effectuant le crossover et la mutation pour créer une nouvelle génération, puis en répétant ces étapes pour plusieurs générations. Elle renvoie le meilleur chromosome trouvé.


In [ ]:
def genetic_algorithm():
    population = generate_population()

    for generation in range(MAX_GENERATIONS):
        # Evaluate fitness of each chromosome in the population
        fitness_scores = [evaluate_fitness(chromosome) for chromosome in population]

        # Select parents for crossover
        parents = select_parents(population)

        # Perform crossover and mutation to create new generation
        new_population = []
        for i in range(0, len(parents),2):
            parent1, parent2 = parents[i], parents[i + 1]
            offspring1, offspring2 = crossover(parent1, parent2)
            mutate(offspring1)
            mutate(offspring2)
            new_population.extend([offspring1, offspring2])

        # Replace the old population with the new generation
        parents.extend(new_population)
        population = parents

    # Return the best chromosome after all generations
    best_chromosome = min(population, key=lambda chromosome: evaluate_fitness(chromosome))
    return best_chromosome

# Main program
best_schedule = genetic_algorithm()
print(best_schedule)
print("Overlapping hours:", evaluate_fitness(best_schedule))

### L'algorithme présenté est une approche d'algorithme génétique pour la planification de séances dans des intervalles de temps prédéfinis. Il utilise une population de chromosomes représentant les différentes combinaisons d'emplois du temps possibles. L'objectif est de minimiser le chevauchement des séances et d'optimiser l'utilisation des créneaux horaires disponibles.